In [16]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd
import csv
#import psycopg2

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
my_index = ["ps_trace-2018*"]

my_query = {}

# CHANGE SO IT RETREIVES ONLY UNIQUE hashes.

# sS='UC'
# srcSiteOWDServer = "192.170.227.160"
# srcSiteThroughputServer = "192.170.227.162"

sS='CERN-PROD'
srcSiteOWDServer = "128.142.223.247"
srcSiteThroughputServer = "128.142.223.246"

# dS='IU'
# destSiteOWDServer = "149.165.225.223"
# destSiteThroughputServer = "149.165.225.224"

# dS='UIUC'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.96.9"

# dS='ICCN'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.126.132"

dS='pic'
destSiteOWDServer = "193.109.172.188"
destSiteThroughputServer = "193.109.172.187"
start_date = '20180104T000000Z'
end_date = '20180404T000000Z'
my_src_query = {
    "size":1,
    "_source": {
        "include": [ 'src', 'dest', 'src_host', 'dest_host', 'src_site', 'dest_site' ]
    },
    'query':{
        'bool':{
            'must':[
                {'range': {'timestamp': {'gte': start_date, 'lt': end_date}}},
                #{'term': {'_type': 'traceroute'}},
#                         {'bool':
#                             {'should':[
#                                 {'term': {'src': srcSiteOWDServer}},
#                                 {'term': {'src': srcSiteThroughputServer}},
#                                 {'term': {'src': destSiteOWDServer}},
#                                 {'term': {'src': destSiteThroughputServer}}
#                             ]}
#                         }
#                         ,
#                         {'bool':
#                             {'should':[
#                                 {'term': {'dest': destSiteOWDServer}},
#                                 {'term': {'dest': destSiteThroughputServer}},
#                                 {'term': {'dest': srcSiteOWDServer}},
#                                 {'term': {'dest': srcSiteThroughputServer}}
#                             ]}
#                         }
            ]

        }
    },
    "aggs": {
        "grouped_by_hash": {
          "terms": {  "field": "hash", "size":10000 }, #
          "aggs": {
              "top_hash_hits": {
                  "top_hits": {
                      "sort": [ { "_score": { "order": "desc" } } ],
                      "size": 1
                  }
              }
          }
       }
    }
}

src_results = es.search(body=my_src_query, index=my_index, request_timeout=12000)
print(len(src_results))
ipv4_dict = {}
ipv6_dict = {}
ipv4_dict_site = {}
ipv6_dict_site = {}
src_data_size = len(src_results['aggregations']['grouped_by_hash']['buckets'])
src_lists = []
for i in range(0, src_data_size):
    rt_src = src_results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['src']
    rt_dest = src_results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['dest']
    src_name = src_results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['src_host']
    dest_name = src_results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['dest_host']
    if 'src_site' in src_results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source'].keys():
        src_site = src_results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['src_site']
    else:
        src_site = 'missing'
    if 'dest_site' in src_results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source'].keys():
        dest_site = src_results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['dest_site']
    else:
        dest_site = 'missing'
    if ':' in rt_src:
        if src_name not in ipv6_dict.keys():
            ipv6_dict[src_name] = rt_src
            if src_name not in ipv6_dict_site.keys():
                ipv6_dict_site[src_name] = src_site
            if ipv6_dict_site[src_name] == 'missing':
                ipv6_dict_site[src_name] = src_site
    else:
        if src_name not in ipv4_dict.keys():
            ipv4_dict[src_name] = rt_src
            if src_name not in ipv4_dict_site.keys():
                ipv4_dict_site[src_name] = src_site
            if ipv4_dict_site[src_name] == 'missing':
                ipv4_dict_site[src_name] = src_site
    if ':' in rt_dest:
        if dest_name not in ipv6_dict.keys():
            ipv6_dict[dest_name] = rt_dest
            if dest_name not in ipv6_dict_site.keys():
                ipv6_dict_site[dest_name] = dest_site
            if ipv6_dict_site[dest_name] == 'missing':
                ipv6_dict_site[dest_name] = dest_site
    else:
        if dest_name not in ipv4_dict.keys():
            ipv4_dict[dest_name] = rt_dest
            if dest_name not in ipv4_dict_site.keys():
                ipv4_dict_site[dest_name] = dest_site
            if ipv4_dict_site[dest_name] == 'missing':
                ipv4_dict_site[dest_name] = dest_site
total = 0
missing = 0
#print(ipv4_dict)
if len(ipv4_dict) > len (ipv6_dict):
    total = len(ipv4_dict)
    for y in ipv4_dict_site.keys():
        if ipv4_dict_site[y] == "missing":
            missing += 1
    for x in ipv6_dict.keys():
        if x not in ipv4_dict.keys():
            total += 1
            if ipv6_dict_site[x] == "missing":
                missing += 1
else:
    total = len(ipv6_dict)
    for y in ipv6_dict_site.keys():
        if ipv6_dict_site[y] == "missing":
            missing += 1
    for x in ipv4_dict.keys():
        if x not in ipv6_dict.keys():
            total += 1
            if ipv4_dict_site[x] == "missing":
                missing += 1
print(len(ipv4_dict))
print(len(ipv6_dict))
print(missing)
print(total)

5
412
63
233
425
